In [1]:
import webscraper_seohyun as scraper
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [2]:
scraper.scrap_song_page(2117748)

{'id': 2117748,
 'name': 'Lollipop',
 'singers': [198094, 406712],
 'genre': '댄스',
 'num_like': 29246,
 'num_comment': 143}

In [4]:
def extract_album_page(source):
    soup = BeautifulSoup(source, 'lxml')
    name = soup.select_one('div.song_name').text.split('\n')[2].strip()
    artists = soup.select('div.artist>a.artist_name>span')
    artists = [x.text for x in artists if x.text != '']
    release_date = soup.select_one('div.meta>dl>dd:nth-child(2)').text
    num_like = int(soup.select_one('#d_like_count').text.replace(',', ''))
    rating = float(soup.select_one('#gradPointLayer').text)
    num_rating = int(soup.select_one('#gradCountLayer').text.replace(',', '')[:-1])
    num_comments = int(soup.select_one('#revCnt').text.replace(',', '')[:-1])
    
    song_list = [a['href'] for a in soup.select('a.song_info')]
    song_list = [x[11:] for x in song_list]
    
    return name, artists, release_date, num_like, rating, num_rating, num_comments, song_list

def extract_artist_page(source):
    soup = BeautifulSoup(source, 'lxml')
    name = soup.select_one('p.title_atist').text[5:]
    
    if '\xa0' in name:
        name = name.split('\xa0')[0]
    
    num_fans = int(soup.select_one('#d_like_count').text.replace(',', '')) 
    columns = soup.select('dt')
    debut_date, artist_type, company_name = None, None, None
    for c in columns:
        if c.text == '데뷔':
            debut = list(c.find_next_sibling().children)[1].text
            if '\n' in debut:
                debut_date = debut.split()[0]
            else:
                debut_date = debut
        elif c.text == '활동유형':
            artist_type = c.find_next_sibling().text
        elif c.text == '소속사':
            company_name = c.find_next_sibling().text
    
    return name, artist_type, company_name, debut_date, num_fans

year = 2004
"""
driver = scraper.navigate_to_year_chart(year)
song_details = driver.find_elements_by_class_name('btn_icon_detail')
scripts = [s[11:] for s in [a.get_attribute('href') for a in song_details]]
songs = []
for s in scripts:
    driver.execute_script(s)    
    song_name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'song_name'))).text.strip()
    genre = driver.find_element_by_css_selector('div.meta > dl > dd:nth-child(6)').text.strip()
    review_cnt = driver.find_element_by_id('revCnt').text.replace(',', '').replace('개', '')
    songs.append((song_name, genre, review_cnt))

driver.quit()
"""
driver = scraper.navigate_to_year_chart(year)

artist_album_details = driver.find_elements_by_css_selector('a.fc_mgray')
scripts = [s[11:] for s in [a.get_attribute('href') for a in artist_album_details]]
artist = []
albums = []

for s in scripts:
    driver.execute_script(s)
    if 'Album' in s:
        albums.append(extract_album_page(driver.page_source))
        break
    elif 'Artist' in s:
        artist.append(extract_artist_page(driver.page_source))
    


"""
for a in set(songs):
    print(a)

for a in set(artist):
    print(a)
"""
for a in albums:
    print(a)

driver.quit()

('Great Expectation 조PD Pt. 2: Love And Life', ['조pd (ZoPD)'], '2004.03.23', 777, 4.6, 665, 197, ["melon.link.goSongDetail('518616');", "melon.link.goSongDetail('518617');", "melon.link.goSongDetail('518618');", "melon.link.goSongDetail('518619');", "melon.link.goSongDetail('518620');", "melon.link.goSongDetail('518621');", "melon.link.goSongDetail('518622');", "melon.link.goSongDetail('518623');", "melon.link.goSongDetail('518624');", "melon.link.goSongDetail('518625');", "melon.link.goSongDetail('518626');", "melon.link.goSongDetail('518627');", "melon.link.goSongDetail('518628');"])


In [3]:
artist

[('조pd (ZoPD)', '솔로', '스타덤', '1991.01', 1362)]